In [ ]:
from __future__ import print_function
import tensorflow.python #.keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Input, Reshape
from tensorflow.python.keras.layers import Conv2D, UpSampling2D, BatchNormalization
from tensorflow.python.keras.layers import LeakyReLU
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras import backend as K
import numpy as np
import pickle

In [ ]:
with open('data.pkl', 'rb') as f:
    x_train = pickle.load(f)
x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
m = 4; n = 1; k = 35;
X_train = np.zeros((x_train.shape[0]-k-m, x_train.shape[1], x_train.shape[2], x_train.shape[3], m));
Y_train = np.zeros((x_train.shape[0]-k-m, x_train.shape[1], x_train.shape[2], x_train.shape[3]));
X_test = np.zeros((k, x_train.shape[1], x_train.shape[2], x_train.shape[3], m));
Y_test = np.zeros((k, x_train.shape[1], x_train.shape[2], x_train.shape[3]));
for i in range(x_train.shape[0]-m-k):
    for j in range(m):
        X_train[i,:,:,:,j] = x_train[i+j]
    Y_train[i] = x_train[i+m]
for i in range(x_train.shape[0]-m-k, x_train.shape[0]-m):
    for j in range(m):
        X_test[i-x_train.shape[0]+m+k,:,:,:,j] = x_train[i+j]
    Y_test[i-x_train.shape[0]+m+k] = x_train[i+m]

X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]*m])
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3]*m])

In [ ]:
input_d = Input(shape = (144, 256, 3*m,))
layerd1 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (input_d)
layerd1 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd1)))
layerd1 = UpSampling2D()(layerd1)

layerd2 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd1)
layerd2 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd2)))
layerd2 = UpSampling2D()(layerd2)

layerd3 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd2)
layerd3 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd3)))
layerd3 = UpSampling2D()(layerd3)

output = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerd3)

model_d = Model(inputs=input_d, outputs=output);

In [ ]:
model_d.summary()

In [ ]:
model_d.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy'])

In [ ]:
stopper = EarlyStopping(monitor='acc',
                              min_delta=0,
                              patience=0,
                              verbose=1, mode='auto')
model_d.fit(X_train, Y_train, 
          batch_size=8, epochs=10, callbacks=[stopper], verbose=1)

In [ ]:
Yhat_test = model_d.predict(X_test)

In [ ]:
import numpy as np
import math

def psnr(img1, img2):
    mse = np.mean((img1-img2)**2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
print(psnr(Yhat_test[8,:,:,:],Y_test[8,:,:,:]))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(Yhat_test[8,:,:,:])